# Chrun Prediction

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,LabelEncoder
import pickle

In [3]:
df = pd.read_csv("Churn_Modelling.csv")

In [4]:
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


**Preprocess the data**

In [5]:
df = df.drop(['RowNumber','CustomerId','Surname'],axis=1)

In [6]:
df.head(3)

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1


**Encode**

In [7]:
label_encoder_gender = LabelEncoder()
df['Gender'] = label_encoder_gender.fit_transform(df['Gender'])

In [8]:
df.head(3)

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,0,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,0,41,1,83807.86,1,0,1,112542.58,0
2,502,France,0,42,8,159660.80,3,1,0,113931.57,1


Onehot encode Geography

In [9]:
from sklearn.preprocessing import OneHotEncoder

In [10]:
onehot_geo = OneHotEncoder(sparse_output=False)
geo_encoder = onehot_geo.fit_transform(df[['Geography']])
geo_encoder

array([[1., 0., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       ...,
       [1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.]])

In [11]:
onehot_geo.get_feature_names_out()

array(['Geography_France', 'Geography_Germany', 'Geography_Spain'],
      dtype=object)

In [12]:
encoded_geography = pd.DataFrame(geo_encoder,columns=onehot_geo.get_feature_names_out(['Geography']))
encoded_geography.head(3)

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0


Combine Columns

In [13]:
data = pd.concat([df,encoded_geography],axis=1)
data.drop('Geography',axis=1,inplace=True)
data.head(3)

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0


Split data

In [14]:
X = data.drop('Exited',axis=1)
y = data['Exited']
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)

Scale data

In [15]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.fit_transform(X_test)

Save encoder and scaler

In [16]:
with open('label_encoder_gender.pkl','wb') as file:
    pickle.dump(label_encoder_gender,file)
with open('onehot_encoder_geography.pkl','wb') as file:
    pickle.dump(onehot_geo,file)
with open('scale.pkl','wb') as file:
    pickle.dump(scaler,file)

Train ANN

In [17]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Input,Activation,BatchNormalization,Dropout
from tensorflow.keras.callbacks import EarlyStopping,TensorBoard
import datetime

In [18]:
import logging
import os
import sys
import warnings
from urllib.parse import urlparse

import dagshub
import numpy as np
import pandas as pd
from sklearn.linear_model import ElasticNet
from sklearn.model_selection import train_test_split
import webbrowser
import mlflow
import mlflow.sklearn
from mlflow.models import infer_signature

In [19]:
MLFLOW_TRACKING_URI = "https://dagshub.com/arjunravi726/Customer-Churn-Prediction.mlflow"

In [20]:
TOKEN = dagshub.auth.get_token()
os.environ["MLFLOW_TRACKING_USERNAME"] = "arjunravi726"
os.environ["MLFLOW_TRACKING_PASSWORD"] = TOKEN

Accessing as arjunravi726

In [21]:
webbrowser.open(MLFLOW_TRACKING_URI)

True

In [22]:
mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)

In [23]:
dagshub.init(repo_name='Customer-Churn-Prediction',repo_owner='arjunravi726',mlflow=True)

Initialized MLflow to track repo "arjunravi726/Customer-Churn-Prediction"

Repository arjunravi726/Customer-Churn-Prediction initialized!

In [24]:
tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme

In [25]:
tracking_url_type_store

'https'

Build ANN model

In [26]:
X.shape[1]

12

In [27]:
model = Sequential([
    Input(shape=(X.shape[1],)),
    BatchNormalization(),
    Dense(64),
    BatchNormalization(),
    Activation('relu'),
    Dropout(rate=0.1),
    Dense(32),
    BatchNormalization(),
    Activation('relu'),
    Dropout(rate=0.1),
    Dense(1, activation='sigmoid')
])

In [28]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ batch_normalization             │ (None, 12)             │            48 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 64)             │           832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 64)             │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation (Activation)         │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 32)             │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_1 (Activation)       │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,377 (13.19 KB)

 Trainable params: 3,161 (12.35 KB)

 Non-trainable params: 216 (864.00 B)

In [29]:
from tensorflow.keras.callbacks import Callback

In [30]:
class CustomCallbacks(Callback):
    def on_epoch_end(self,epoch,logs=None):
        if logs['val_loss'] < 0.5 or logs['val_accuracy'] > 0.9:
            self.model.stop_training = True

In [31]:
optimizer = tf.keras.optimizers.Adam(learning_rate=0.01)

In [32]:
model.compile(optimizer=optimizer,loss='binary_crossentropy',metrics=['accuracy'])

In [33]:
client = mlflow.MlflowClient()
registered_models = client.search_registered_models()
for model in registered_models:
    print(model.name)

Set up Tensorboard

In [34]:
log_dir = "logs/fit" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

In [35]:
tensorflow_callback = TensorBoard(log_dir=log_dir,histogram_freq=1)

Early Stopping

In [36]:
early_stopping = EarlyStopping(monitor='val_loss',patience=5,restore_best_weights=True)

In [37]:
history = model.fit(X_train_scaled,y_train,validation_data=(X_test_scaled,y_test),epochs=100,callbacks=[tensorflow_callback,CustomCallbacks()])

Epoch 1/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 8s 9ms/step - accuracy: 0.8017 - loss: 0.4383 - val_accuracy: 0.8295 - val_loss: 0.3970
Epoch 2/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.8400 - loss: 0.3739 - val_accuracy: 0.8480 - val_loss: 0.3600
Epoch 3/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.8518 - loss: 0.3621 - val_accuracy: 0.8525 - val_loss: 0.3525
Epoch 4/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.8586 - loss: 0.3494 - val_accuracy: 0.8620 - val_loss: 0.3421
Epoch 5/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.8526 - loss: 0.3647 - val_accuracy: 0.8620 - val_loss: 0.3387
Epoch 6/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.8573 - loss: 0.3425 - val_accuracy: 0.8555 - val_loss: 0.3447
Epoch 7/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.8529 - loss: 0.3571 - val_accuracy: 0.8650 - val_loss: 0.3379
Epoch 8/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step - accuracy: 0.8565 - loss: 0.3467 - val_accu

In [38]:
# mlflow.sklearn.log_model(model)

In [39]:
import mlflow
# with mlflow.start_run():
#   # Your training code here...
#   mlflow.log_metric('train_acc', history.history['accuracy'][-1])
#   mlflow.log_param('val_acc',history.history['val_accuracy'][-1])
#   mlflow.log_param('val_loss',history.history['val_loss'][-1])
#   mlflow.log_param('loss',history.history['loss'][-1])

🏃 View run unique-mink-662 at: https://dagshub.com/arjunravi726/Customer-Churn-Prediction.mlflow/#/experiments/0/runs/705cf0583ab24c84a970db7ed2d78ba3
🧪 View experiment at: https://dagshub.com/arjunravi726/Customer-Churn-Prediction.mlflow/#/experiments/0


In [75]:
from dagshub import dagshub_logger

In [69]:
imput_shape = X.shape[1]

In [86]:
save_path = "model"

In [89]:
with dagshub_logger() as logger,mlflow.start_run():
    # Log hyperparameters
    mlflow.log_param("input_shape", imput_shape)
    mlflow.log_param("optimizer", "adam")
    mlflow.log_param("loss_function", "binary_crossentropy")
    mlflow.log_param("metrics", "accuracy")

    # Log metrics
    for epoch, metrics in enumerate(history.history['val_accuracy']):
        logger.log_metrics({"val_accuracy": metrics}, epoch=epoch)
        mlflow.log_metric("val_accuracy", metrics, step=epoch)
    for epoch, metrics in enumerate(history.history['val_loss']):
        logger.log_metrics({"val_loss": metrics}, epoch=epoch)
        mlflow.log_metric("val_loss", metrics, step=epoch)
    for epoch, metrics in enumerate(history.history['accuracy']):
        logger.log_metrics({"accuracy": metrics}, epoch=epoch)
        mlflow.log_metric("accuracy", metrics, step=epoch)
    for epoch, metrics in enumerate(history.history['loss']):
        logger.log_metrics({"loss": metrics}, epoch=epoch)
        mlflow.log_metric("loss", metrics, step=epoch)

    # Save and log the model
    model.save("ann_model.keras")
    mlflow.tensorflow.log_model(model, save_path,
        registered_model_name="ANN_Model_Registry")

    # Log the training history as an artifact
    with open("training_history.txt", "w") as f:
        f.write(str(history.history))
    mlflow.log_artifact("training_history.txt")

2025/01/19 09:49:20 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.
2025/01/19 09:49:58 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
Successfully registered model 'ANN_Model_Registry'.
2025/01/19 09:50:05 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: ANN_Model_Registry, version 1
Created version '1' of model 'ANN_Model_Registry'.


🏃 View run spiffy-snail-719 at: https://dagshub.com/arjunravi726/Customer-Churn-Prediction.mlflow/#/experiments/0/runs/7878d91c42cd4f608f32632b9db5ea76
🧪 View experiment at: https://dagshub.com/arjunravi726/Customer-Churn-Prediction.mlflow/#/experiments/0


In [84]:
mlflow.end_run()

🏃 View run kindly-slug-328 at: https://dagshub.com/arjunravi726/Customer-Churn-Prediction.mlflow/#/experiments/0/runs/7c6bfa93426f4279bab4f414fc45a1c5
🧪 View experiment at: https://dagshub.com/arjunravi726/Customer-Churn-Prediction.mlflow/#/experiments/0


In [41]:
# import keras
# keras.saving.save_model(model, 'model.keras')

In [42]:
model.save('model.h5')

Load Tensorboard Extension

In [43]:
%load_ext tensorboard

In [44]:
!kill 6460

'kill' is not recognized as an internal or external command,
operable program or batch file.


In [45]:
%tensorboard --logdir logs/fit20241228-160457

Reusing TensorBoard on port 6006 (pid 12184), started 12:13:54 ago. (Use '!kill 12184' to kill it.)

Prediction

In [46]:
from tensorflow.keras.models import load_model

In [47]:
loaded_model = load_model('model.h5')

In [48]:
with open('label_encoder_gender.pkl','rb') as file:
    label_encoder_gender = pickle.load(file)

In [49]:
with open('onehot_encoder_geography.pkl','rb') as file:
    onehot_geo = pickle.load(file)

In [50]:
with open('scale.pkl','rb') as file:
    scaler = pickle.load(file)

In [51]:
input_data = {
    'CreditScore':600,
    'Geography': 'France',
    'Gender':'Male',
    'Age':40,
    'Tenure':3,
    'Balance':60000,
    'NumOfProducts':2,
    'HasCrCard': 1,
    'IsActiveMember':1,
    'EstimatedSalary':50000
}

In [52]:
df = pd.DataFrame([input_data])

In [53]:
label_encoder_gender.transform(df['Gender'])

array([1])

In [54]:
df['Gender'] = label_encoder_gender.transform(df['Gender'])

In [55]:
df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,1,40,3,60000,2,1,1,50000


In [56]:
geo_encoded= onehot_geo.transform([df['Geography']])
geo_encoded_df = pd.DataFrame(geo_encoded,columns=onehot_geo.get_feature_names_out())

d:\BroCamp\Deep Learning\ANN\venv\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


In [57]:
geo_encoded_df

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [58]:
df = pd.concat([df,geo_encoded_df],axis=1)

In [59]:
df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,France,1,40,3,60000,2,1,1,50000,1.0,0.0,0.0


In [60]:
df.drop('Geography',axis=1,inplace=True)

In [61]:
df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,1,40,3,60000,2,1,1,50000,1.0,0.0,0.0


In [62]:
df_scaled = scaler.transform(df)

In [63]:
df_scaled

array([[-0.47154541,  0.90911166,  0.09477172, -0.69844549, -0.29010416,
         0.80510537,  0.63367318,  0.95214374, -0.84805047,  0.98019606,
        -0.57581067, -0.56349184]])

In [64]:
prediction_prob = loaded_model.predict(df_scaled)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 422ms/step


In [65]:
if prediction_prob > 0.5:
    print('The Customer is likely to churn')
else:
    print('The Customer is not likely to churn')

The Customer is not likely to churn


In [66]:
onehot_geo.categories_

[array(['France', 'Germany', 'Spain'], dtype=object)]

Intergrate with streamlit